In [1]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [2]:
# Test Ollama

from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': 'What is the capital of France? Can you also give me that city\'s history?',
  },
])
print(response['message']['content'])

The capital of France is Paris. Here's a brief overview of its rich and fascinating history:

**Ancient Times (52 BC - 486 AD)**

Paris was founded by the Celtic tribe known as the Parisii in the 3rd century BC. The Romans conquered the city in 52 BC and renamed it Lutetia Parisiorum, which translates to "Lutece of the Parisians." During this period, Paris became an important center for trade, culture, and politics.

**Middle Ages (486 - 1492)**

In the 5th century AD, Paris was conquered by the Franks, a Germanic tribe. The city became an important center for Christianity, with the construction of several churches and monasteries. In the 9th century, Charlemagne, King of the Franks, made Paris his capital.

**Renaissance and Enlightenment (1492 - 1789)**

During the Renaissance, Paris became a hub for art, literature, and science. The city was home to prominent figures such as Leonardo da Vinci, Michelangelo, and Voltaire. The French Revolution in 1789 marked a significant turning poi

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader

# --- Load the text and split into chunks ---
loader = UnstructuredMarkdownLoader("output.md")
data = loader.load()

headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(data[0].page_content)

# Add a second layer of splitting to stay under 40KB
# 2000 characters is roughly 2-3KB, well within Pinecone's 40KB limit
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=500
)

# Apply the second splitter to the header-splits, preserving metadata on each chunk
final_docs = text_splitter.split_documents(md_header_splits)

In [6]:
!pip install "torch>=2.0" "transformers>=4.40" "sentence-transformers>=3.0" "langchain-huggingface" "langchain-pinecone" "python-dotenv"

  Obtaining dependency information for torch>=2.0 from https://files.pythonhosted.org/packages/6e/ab/07739fd776618e5882661d04c43f5b5586323e2f6a2d7d84aac20d8f20bd/torch-2.9.1-cp312-none-macosx_11_0_arm64.whl.metadata
  Using cached torch-2.9.1-cp312-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Obtaining dependency information for transformers>=4.40 from https://files.pythonhosted.org/packages/6a/6b/2f416568b3c4c91c96e5a365d164f8a4a4a88030aa8ab4644181fdadce97/transformers-4.57.3-py3-none-any.whl.metadata
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Obtaining dependency information for sentence-transformers>=3.0 from https://files.pythonhosted.org/packages/40/d0/3b2897ef6a0c0c801e9fecca26bcc77081648e38e8c772885ebdd8d7d252/sentence_transformers-5.2.0-py3-none-any.whl.metadata
  Using cached sentence_transformers-5.2.0-py3-none-any.whl.metadata (16 kB)
  Obtaining dependency information for langchain-huggingface from https://files.pythonhosted.org/packages/81/ce/5

In [8]:
# --- For import issues with torch library ---
import torch.optim.lr_scheduler as lr_scheduler
if not hasattr(lr_scheduler, "LRScheduler"):
    lr_scheduler.LRScheduler = lr_scheduler._LRScheduler

import os
import time
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

# --- Pull API Keys ---
load_dotenv("info.env")
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Model matches 384 dimensions
embed_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embed_model_name)

# Create index
index_name = "rag-test-index"

# Force a clean start if the dimensions are wrong
if index_name in [idx.name for idx in pc.list_indexes()]:
    desc = pc.describe_index(index_name)
    if desc.dimension != 384:
        print(f"Index has wrong dimension ({desc.dimension}). Deleting...")
        pc.delete_index(index_name)
        time.sleep(5) # Wait for Pinecone to clear

# Create the correct 384-dimension index
if index_name not in [idx.name for idx in pc.list_indexes()]:
    print(f"Creating new 384-dimension index...")
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )

# Store final_docs
vector_store = PineconeVectorStore.from_documents(
    final_docs,
    embeddings,
    index_name=index_name,
)
print(f"Upsert successful! Total chunks created: {len(final_docs)}")

Upsert successful! Total chunks created: 38


In [9]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Initialize the LLM using Ollama
llm = ChatOllama(
    model="llama3.2",
    temperature=0,
)

# --- Create the RAG Pipeline ---
template = """Answer the question based only on the following context:
{context}

Question: {input}
"""
prompt = ChatPromptTemplate.from_template(template)

# Define the "Chain" using the pipe syntax with the Pinecone vector store
retriever = vector_store.as_retriever(earch_kwargs={"k": 7})

rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# --- Run the Pipeline ---
response = rag_chain.invoke("What are David Mitchell's rules of trading stocks, what are some general facts, and what are some important patterns to know?")
print(response)

According to the provided context, David Mitchell's Stock Rules include:

1. Go through his stock rules and keep them by your computer.
2. Setup charts by TRADEway (charts.bytradeway.com).
3. Open a trading account (tradeway.com/tradier).
4. Wire funds (put money in your trading account).
5. Connect charts by TRADEway with Tradier Account.

Some general facts mentioned include:

1. The importance of diversification, which is often overemphasized and can be a myth.
2. The need to have skill-sets when trading stocks, rather than relying on products sold by financial advisors.
3. Quotes from Warren Buffet and Philip Fisher about diversification.

Important patterns to know include:

1. The TRADEway Three Legged Trading Table:
	* Fundamentals: Shows what to buy, including earnings per share, relative price strength, industry group ratings, return on equity, and accumulation/distribution.
	* Technicals: Shows when to buy and sell fundamentally sound stocks, using charts to measure price and